In [1]:
#1
import numpy as np
%pylab 
style.use('ggplot')
rcParams['figure.figsize'] = (7,7)
from numba import jit

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import copy
import networkx as nx

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
def getRandomConnectivity(N, pct_connected):
    #Directed mapping
    M = np.random.rand(N**2).reshape(N,N)
    for i in range(N):
        for j in range(N):
            if (i == j):
                M[i,j] = 0
            else:
                if (M[i,j] < pct_connected):
                    M[i,j] = 1 #i connects to j
                else:
                    M[i,j] = 0
    return M

# alternate version
def getRandomConnectivity2(N, pct_connected):
    # There are N*(N-1)/2 upper diagonal elements
    rand_elems = np.where(np.random.rand(N*(N-1)/2) < pct_connected, 1, 0)
    M = np.zeros((N,N))
    M[np.triu_indices(N,1)] = rand_elems
    return M + M.T

# Original version with bidirectional mapping 
def getRandomConnectivity3(N, pct_connected):
    # There is no self coupling so the diagonal must be zero. 
    # The graph is non directed so M must be symmetric
    M = np.random.rand(N**2).reshape(N,N)
    for i in range(N):
        for j in range(i, N):
            if (i == j):
                M[i,j] = 0
            else:
                if (M[i,j] < pct_connected):
                    M[i,j] = 1
                else:
                    M[i,j] = 0
                M[j,i] = M[i,j]
    return M

In [3]:
def hasKcore(M, k):
    # we'll be destroying the array so make a copy to work with
    X = M.copy()
    while(True):
        cur_num_nodes = X.shape[0]
        s = np.sum(X, 0)
        nodes_to_delete = np.where(s < k)[0]
        if (len(nodes_to_delete) == cur_num_nodes):
            # nothing has at least k connections
            
            X = np.delete(X, nodes_to_delete, axis=0)
            X = np.delete(X, nodes_to_delete, axis=1)
            
            return False
            break
        elif (len(nodes_to_delete) == 0):
            # They all have at least k connections, we've found a kcore
            return True
            break
        else:
            X = np.delete(X, nodes_to_delete, axis=0)
            X = np.delete(X, nodes_to_delete, axis=1)

            
# TODO get Kcore indices 

def largestKcore(M):
    # we can begin the search at the least connected node
    connectivity = np.sum(M,0)
    min_kcore = np.min(connectivity)
    max_kcore = np.max(connectivity)
    if max_kcore == 0:
        return 0
    k = min_kcore
    while (hasKcore(M,k)):
        k = k+1
        
    return k - 1
        
    
    
def largestkcore(M):
    # we can begin the search at the least connected node
    connectivity = np.sum(M,0)
    min_kcore = np.min(connectivity)
    max_kcore = np.max(connectivity)
    if max_kcore == 0:
        return 0
    k = min_kcore
    s=1
    while (s==1):
        
        X = M.copy()
        while(True):
            cur_num_nodes = X.shape[0]
            s = np.sum(X, 0)
            nodes_to_delete = np.where(s < k)[0]
            if (len(nodes_to_delete) == cur_num_nodes):
            # nothing has at least k connections
            
                X = np.delete(X, nodes_to_delete, axis=0)
                X = np.delete(X, nodes_to_delete, axis=1)
            
                s=0
                break
            elif (len(nodes_to_delete) == 0):
            # They all have at least k connections, we've found a kcore
                s=1
                break
            else:
                X = np.delete(X, nodes_to_delete, axis=0)
                X = np.delete(X, nodes_to_delete, axis=1)
        k = k+1
        
    return k - 1

#TODO: Instead of deleting nodes, put 0's on all the connections for the rows and columns that are disconnected
    #Keeps number of all neurons the same so that neuron 3 doesnt become neuron 2 etc. 
    #Look for neurons not in Kcore which are firing and neurons that are in kcore and not firing. 
def kcoreitself(M, k):
    # we'll be destroying the array so make a copy to work with
    X = M.copy()
    
    while(True):
        cur_num_nodes = X.shape[0]
        s = np.sum(X, 0)
        nodes_to_delete = np.where(s < k)[0]
        if (len(nodes_to_delete) == cur_num_nodes):
            
            X = np.delete(X, nodes_to_delete, axis=0)
            X = np.delete(X, nodes_to_delete, axis=1)
            # nothing has at least k connections
            return X
            break
        elif (len(nodes_to_delete) == 0):
            # They all have at least k connections, we've found a kcore
            return X
            break
        else:
            X = np.delete(X, nodes_to_delete, axis=0)
            X = np.delete(X, nodes_to_delete, axis=1)



#here we get the largest connected component of  the graph

def largestcomponent(l,N):
    n=np.random.randint(0,N)
    
    boundary=[n]
    b=1
    comp=[n]
    c=1
   
    while(b>0):
        a=boundary.pop()
        
        oldvertex=int(a)
        
        deg=l[oldvertex,0]
        degree=int(deg)
        #print(degree)
        for j in range(degree):
            newvertex=l[oldvertex,j+1]
            q=comp.count(newvertex)
            #print(newvertex)
            q=int(q)
            #print(q)
            if (q==0):
                comp.append(newvertex)
                boundary.append(newvertex)
                #print(newvertex)
                #print(boundary)
            
        b=len(boundary)
            
    #sizeofcomp=len(comp)
    return(comp)

def matrixOfEdges(M,N):
    N = int(N)
    E=np.zeros(N**2).reshape(N,N)  #prepare array with zeros
    k=np.sum(M,0) #array with degree of each vertex
    for i in range(N):
        a=int(k[i]) #degree of the current vertex
        E[i,0]=a  #we put it to the zero row of matrix of edges
    for i in range(N):
        a=int(k[i])
        q=1
        for j in range(N):
            if (M[j,i]==1):
                E[i,q]=j #all the next elements in current column are number of vertices current vertex is connected to
                q=q+1
    return(E)

In [4]:
def simdyn(M,l,params):
    return simDynamics(M,l,params)

@jit(nopython=True)
def simDynamics(M,l,params):
    N=int(params[0])
    n=int(params[1])
    dt=params[2]
    tauv=params[3]
    tauc=params[4]
    Vstar=params[5]
    Cstar=params[6]
    delV=params[7]
    delC=params[8]
    r=params[9]
    V0=params[10]
    C0=params[11]
    V=zeros(N*n).reshape(N,n)  #now we have potencial and calcium concentration for each neuron
    C=zeros(N*n).reshape(N,n)
    P=zeros(n)  # function P(V), sigmoid
    sump=zeros(n)  #sum of P over the vertices
    avV=zeros(N) #average potencial
    avC=zeros(N)  #average calcium
    #res = int(params[12]) #resulting classification 
    firing_neurons_count = zeros(N*n).reshape(N,n) #Individually count how may neurons are actually firing in this model at a given step N.
    
    for i in range(n):
        V[0,i]=V0 #initial conditions
        C[0,i]=C0 
        if V[0,i] > Vstar:
            firing_neurons_count[0,i] = 1
        avV[0]=n*V0  #we will divide to n later
        avC[0]=n*C0
    for i in range (N-1):
        for j in range(n):
            P[j] = 1/(1+exp((Vstar-V[i,j])*0.2))
        for j in range(n):
            a=int(l[j,0])
            for k in range(a):
                b=int(l[j,k])
                sump[j]=sump[j]+P[b]  #if neuron connected to j-th neuron it participates in sum
        for j in range(n):
            V[i+1,j]=(-V[i,j]/tauv + delV*r*sump[j]*(1/(1+exp(C[i,j]-Cstar))) )*dt + V[i,j]
            if V[i+1,j] > Vstar: 
                firing_neurons_count[i+1,j] = 1 #used to return count of firing neurons later
            C[i+1,j]=(-C[i,j]/tauc + delC*r*sump[j])*dt + C[i,j]
            avC[i+1]=avC[i+1]+C[i+1,j]
            avV[i+1]=avV[i+1]+V[i+1,j]
        for j in range(n):
            sump[j]=0
    for i in range (N):
        avC[i]=avC[i]/n
        avV[i]=avV[i]/n
        
   
    return(avC,avV,firing_neurons_count)

In [5]:
import pdb
#for some reason we need this funky embedding
def simdyn3(M, l, params):
    return simDynamics3(M, l, params)



@jit(nopython=True)
def simDynamics3(M, l, params):  #params is a vector of parameters
    N=int(params[0])
    n=int(params[1])
    dt=params[2]
    tauv=params[3]
    tauc=params[4]
    Vstar=params[5]
    Cstar=params[6]
    delV=params[7]
    delC=params[8]
    r=params[9]
    V0=params[10]
    C0=params[11]
    delVmin = params[12]
    V=zeros(N*n).reshape(N,n)  #now we have potencial and calcium concentration for each neuron
    C=zeros(N*n).reshape(N,n)
    #P=zeros(n)  # function P(V), sigmoid
    P = zeros(N*n).reshape(N,n)
    deltaV=zeros(n)
    sump=zeros(n)  #sum of P over the vertices
    avV=zeros(N) #average potencial
    avC=zeros(N)  #average calcium
    firing_neurons_count = zeros(N*n).reshape(N,n) #Individually count how may neurons are actually firing in this model at a given step N.
    
    for i in range(n):
        V[0,i]=V0 #initial conditions
        C[0,i]=C0 
        if V[0,i] > Vstar: #keep track of and return how many neurons in the network are actually firing  
            firing_neurons_count[0,i] = 1
        avV[0]=n*V0  #we will divide to n later
        avC[0]=n*C0
    for i in range (N-1):
        for j in range(n):
            if (V[i,j]>Vstar): #V[i,j] stores voltage of each neuron instead of returning the average voltage 
                P[i,j]=70
            else:
                P[i,j]=5
            if (C[i,j]<Cstar):
                deltaV[j]=delV #change from 1.2 
            else:
                deltaV[j]=delVmin #changed from .4
        for j in range(n):
            a=int(l[j,0]) #a should give number of firing neighbors 
            for k in range(a):
                #b=int(l[j,k])
                b = int(l[j,k+1])
                sump[j]=sump[j]+P[i,b]  #if neuron connected to j-th neuron it participates in sum
        for j in range(n):
            V[i+1,j]=(-V[i,j]/tauv + deltaV[j]*sump[j])*dt + V[i,j]
            if V[i+1,j] > Vstar: 
                firing_neurons_count[i+1,j] = 1 #used to return count of firing neurons later
            #if ((firing_neurons_count[i,j] == 1) and (firing_neurons_count[i+1,j] == 0)):
            #    print(sump[j])
            C[i+1,j]=(-C[i,j]/tauc + delC*r*sump[j])*dt + C[i,j]
            avC[i+1]=avC[i+1]+C[i+1,j]
            avV[i+1]=avV[i+1]+V[i+1,j]
        for j in range(n):
            sump[j]=0
    for i in range (N):
        avC[i]=avC[i]/n
        avV[i]=avV[i]/n
        
        #plot([avC[i],avC[i+1]],[avV[i],avV[i+1]])    
    return(avC,avV, firing_neurons_count, P, V)

In [6]:
def classify(V, params):
    #Set up some globals 
    
    N = int(params[0])
    n = int(params[1])
    
    #Network uniformly connected but vectorized lets us scale later 
    #for i in range (N/2, N):
    max_V = max(V[int(N/2):N])
    min_V = min(V[int(N/2):N])
   
            
    V_delta = max_V - min_V
    
    if V_delta < 0.01: #changed from 1
        if V[N-1] < 15:
            #params[12] = 2
            return 2
        else:
            #params[12] = 3
            return 3
    else:
        #params[12] = 1
        return 1

In [7]:
def killNeuron(N,M_prime):
    M = copy.deepcopy(M_prime)
    #Randomly pick a neuron to remove
    #x = np.random.randint(N)
    x = 1
    M = np.delete(M,x,axis=0)
    M = np.delete(M,x,axis=1)
    return M

#TODO: Kill N neurons from the network systematically instead of one at a time. 
def killNeuron2(to_cull, M_prime):
    M = copy.deepcopy(M_prime) 
    #Systemically cull certain neuron from the matrix
    x = 0    
    for i in range (int(to_cull)):
        M = np.delete(M,x,axis=0)
        M = np.delete(M,x,axis=1)
    return M

In [210]:
#Phase Diagrams 
N=10000
n=150 #100
dt=0.001
T=zeros(N)
for i in range(N):
    T[i]=i*dt
p = 1/6 #Connectivity rate High connectivity is just mean field so it looks a lot smoother... want to maybe not make it so high

M=getRandomConnectivity(n,p) #args(n,p)
l=matrixOfEdges(M,n)
M_prime = copy.deepcopy(M)
l_prime = copy.deepcopy(l)
#curr_n = n #Current number of neurons in network 
to_cull = 0

tauv=0.01 #0.01
tauc=0.5 #0.5
Vstar=15 #Constant (according to experiments) 15
Cstar=20 #15
V0=18 #18
C0=18 #18

delV= 4. #1.2
delVmin = 0#delV/3
delC= 0.015 #0.015
r= 70 #70

params=zeros(13)
params[0]=N
params[1]=n
params[2]=dt
params[3]=tauv
params[4]=tauc
params[5]=Vstar
params[6]=Cstar
params[7]=delV
params[8]=delC
params[9]=r
params[10]=V0
params[11]=C0
params[12]=delVmin #connenctivity rate from above.


phasen = []
phaseV = []
kCore = []

dV = 0.1
delVmax = 20.

while True:
    C,V,f,P,Voltages = simdyn3(M_prime,l_prime,params)
    check = classify(V,params)
    while True:
        #import pdb; pdb.set_trace()
        params[1] -= 1
        #curr_n = curr_n-1
        to_cull += 1
        M_prime = killNeuron2(to_cull, M)
        l_prime = matrixOfEdges(M_prime, params[1])
        if params[1] <= 1:
            params[7] += dV
            params[12] = params[7]/3
            params[1] = n
            to_cull = 0
            M_prime = copy.deepcopy(M)
            l_prime = copy.deepcopy(l)
            #curr_n = n
            break
        C,V,f,P,Voltages = simdyn3(M_prime,l_prime,params)
        check2 = classify(V,params)
        if check2 != check:
            print("delV: ", params[7])
            print("transition from", check, "to", check2)
            #print("Kcore: ", largestKcore(M_prime), "Predicted:",floor(Vstar/(tauv*70*params[12])))
            print("Kcore: ", largestKcore(M_prime), "Predicted:",Vstar/(tauv*70*params[12]))
            phasen.append(params[1])
            phaseV.append(params[7])
            kCore.append(largestKcore(M_prime))
            params[7] += dV
            params[12] = params[7]/3
            if params[1] < n-3:
                to_cull -= 3
                params[1] += 3 #Changed from params[1] = n to track boundary 
                M_prime = killNeuron2(to_cull, M)
                l_prime = matrixOfEdges(M_prime, params[1])
            else:
                to_cull = 0
                params[1] = n 
                M_prime = copy.deepcopy(M)
                l_prime = copy.deepcopy(l)
                
            break

    if params[7] > delVmax:
        break
print('loop complete')


delV:  4.1
transition from 3 to 2
Kcore:  15.0 Predicted: 15.6794425087
delV:  4.2
transition from 3 to 2
Kcore:  15.0 Predicted: 15.306122449
delV:  4.3
transition from 3 to 2
Kcore:  14.0 Predicted: 14.950166113
delV:  4.4
transition from 3 to 2
Kcore:  14.0 Predicted: 14.6103896104
delV:  4.5
transition from 3 to 2
Kcore:  14.0 Predicted: 14.2857142857
delV:  4.6
transition from 3 to 2
Kcore:  13.0 Predicted: 13.9751552795
delV:  4.7
transition from 3 to 2
Kcore:  13.0 Predicted: 13.6778115502
delV:  4.8
transition from 3 to 2
Kcore:  13.0 Predicted: 13.3928571429
delV:  4.9
transition from 3 to 2
Kcore:  13.0 Predicted: 13.1195335277
delV:  5.0
transition from 3 to 2
Kcore:  12.0 Predicted: 12.8571428571
delV:  5.1
transition from 3 to 2
Kcore:  12.0 Predicted: 12.6050420168
delV:  5.2
transition from 3 to 2
Kcore:  12.0 Predicted: 12.3626373626
delV:  5.3
transition from 3 to 2
Kcore:  12.0 Predicted: 12.1293800539
delV:  5.4
transition from 3 to 2
Kcore:  11.0 Predicted: 11.90476

delV:  15.7
transition from 3 to 2
Kcore:  3.0 Predicted: 4.09463148317
delV:  15.8
transition from 3 to 2
Kcore:  3.0 Predicted: 4.06871609403
delV:  15.9
transition from 3 to 2
Kcore:  3.0 Predicted: 4.04312668464
delV:  16.0
transition from 3 to 2
Kcore:  3.0 Predicted: 4.01785714286
delV:  16.1
transition from 3 to 2
Kcore:  3.0 Predicted: 3.99290150843
delV:  16.2
transition from 3 to 2
Kcore:  3.0 Predicted: 3.96825396825
delV:  16.3
transition from 3 to 2
Kcore:  3.0 Predicted: 3.94390885188
delV:  16.4
transition from 3 to 2
Kcore:  3.0 Predicted: 3.91986062718
delV:  16.5
transition from 3 to 2
Kcore:  3.0 Predicted: 3.8961038961
delV:  16.6
transition from 3 to 2
Kcore:  3.0 Predicted: 3.87263339071
delV:  16.7
transition from 3 to 2
Kcore:  3.0 Predicted: 3.8494439692
delV:  16.8
transition from 3 to 2
Kcore:  3.0 Predicted: 3.82653061224
delV:  16.9
transition from 3 to 2
Kcore:  3.0 Predicted: 3.80388841927
delV:  17.0
transition from 3 to 2
Kcore:  3.0 Predicted: 3.781512

In [212]:
plt.plot(phaseV, phasen)

In [61]:
plt.plot(C,V)

In [217]:
#Looking at k-core size versus the actual number of firing neurons in the network 
N=10000
n=130 #150
dt=0.001
T=zeros(N)
for i in range(N):
    T[i]=i*dt
p = 1 #Connectivity rate High connectivity is just mean field so it looks a lot smoother... want to maybe not make it so high

M=getRandomConnectivity(n,p) #args(n,p)
M_T = transpose(M)
l1=matrixOfEdges(M,n)
l2=matrixOfEdges(M_T,n)
#M_prime = copy.deepcopy(M_T)
#l_prime = copy.deepcopy(l)

tauv=0.01 #0.01
tauc=0.5 #0.5
Vstar=15 #Constant (according to experiments) 15
Cstar=20 #15
V0=30 #18
C0=30 #18

delV= 1.2 #1.2
delVmin = delV/3
delC= 0.015 #0.015
r= 70 #70

#M_prime = killNeuron2(n-cull_to, M_T)
#l_prime = matrixOfEdges(M_prime, cull_to)

params=zeros(13)
params[0]=N
params[1]=n #instead of actual n, cull neurons til desired value
params[2]=dt
params[3]=tauv
params[4]=tauc
params[5]=Vstar
params[6]=Cstar
params[7]=delV
params[8]=delC
params[9]=r
params[10]=V0
params[11]=C0
params[12]=delVmin #connenctivity rate from above.


#C,V,neuron_counts = simdyn3(M_prime,l_prime,params)
C,V,neuron_counts, P, Voltages = simdyn3(M,l1,params)


#C,V,neuron_counts = simdyn(M, l1, params)
itself = kcoreitself(M,ceil(Vstar/(tauv*delVmin*r)))
plt.plot(C,V)

#plt.plot(C,V)

In [159]:
out = Vstar/(tauv*delVmin*r)

print("Classification:",classify(V,params))
print("Largest kcore:", largestKcore(M))
print("Predicted k: ",out)
print("Shape of kcore (gives kcore size):",itself.shape)
print("Number of neurons total in network:",params[1])
print("Number of firing neurons:",sum(neuron_counts[N-1]))

Classification: 1
Largest kcore: 50.0
Predicted k:  53.57142857142857
Shape of kcore (gives kcore size): (0, 0)
Number of neurons total in network: 80.0
Number of firing neurons: 0.0


In [154]:
C,V,firing_neurons = simdyn(M, l1, params)
plt.plot(C,V)

In [161]:
G = nx.DiGraph()
H = nx.DiGraph()
G.add_nodes_from(range(0,n))
H.add_nodes_from(range(0,n))
for i in range(n):
    for j in range(n):
        if M[i,j] == 1: #and neuron_counts[N-1,j] == 1:
            #if neuron_counts[N-1,j] == 1:
            if neuron_counts[N-1,i] == 1:
                H.add_edge(i,j,firing=True)
            else:
                G.add_edge(i,j,firing=False)

In [162]:
plt.title("%d Neurons in network"%G.order())
nx.draw_circular(H,node_size=20,width=0.3,node_color='black', edge_color='red',with_labels=True)
nx.draw_circular(G,node_size=20,width=0.3,edge_color='black',node_color='black')
plt.savefig('neuron_connectivity.png')

### Mean Field Analysis with Step Function 

In [37]:
#Step Function Toy Model 

@jit(nopython=True)
def mean_field_dynamics(params):
    N = int(params[0])
    n = int(params[1]) 
    dt = params[2]
    tauv = params[3]
    tauc = params[4] 
    Vstar = params[5] 
    Cstar = params[6]
    delV = params[7]
    delC = params[8]
    r_1 = params[9]
    r_0 = params[10]
    V_0 = params[11]
    C_0 = params[12]
    
    V=zeros(N*n).reshape(N,n)  #now we have potencial and calcium concentration for each neuron
    C=zeros(N*n).reshape(N,n)
    
    firing_neurons_count = zeros(N*n).reshape(N,n) #Individually count how may neurons are actually firing in this model at a given step N.
    
    avV=zeros(N) #average potencial
    avC=zeros(N)  #average calcium
    
    for i in range(n):
        V[0,i]=V_0 #initial conditions
        C[0,i]=C_0
        if V[0,i] > Vstar: #keep track of and return how many neurons in the network are actually firing  
            firing_neurons_count[0,i] = 1
        avV[0]=n*V0  #we will divide to n later
        avC[0]=n*C0
    
    for i in range(N - 1): 
        for j in range(n):
            if V[i,j] > Vstar: 
                r = r_1
            else:
                r = 0

            if C[i,j] < Cstar: 
                dV = delV 
            else: 
                dV = 0

            V[i+1,j] = (-V[i,j]/tauv + n*dV*(r + r_0)) * dt + V[i,j]
            if V[i+1,j] > Vstar: 
                firing_neurons_count[i+1,j] = 1
            C[i+1,j] = (-C[i,j]/tauc + n*delC*(r + r_0)) * dt + C[i,j]
            avC[i+1]=avC[i+1]+C[i+1,j]
            avV[i+1]=avV[i+1]+V[i+1,j]
            
    for i in range (N):
        avC[i]=avC[i]/n
        avV[i]=avV[i]/n    
    
    return avC, avV, firing_neurons_count 
    
    

N = 1000 #Number of Steps 
dt = 0.001

T = zeros(N)
#V = zeros(N) 
#C = zeros(N)

#V[0] = 18
#C[0] = 18

V_0, C_0 = 18, 18

tauv = 0.01
tauc = 0.5

Vstar = 15#15
Cstar = 20#20

n = 100 #100

delV = 5
delC = 0.015
r_1 = 70 #3000
r_0 = 5 #10

conditions = tauc*delC*r_1 > Cstar and tauc*delC*r_0 < Cstar and delV*tauv*r_0 > Vstar

params = zeros(13)
params[0] = N
params[1] = n
params[2] = dt 
params[3] = tauv 
params[4] = tauc 
params[5] = Vstar
params[6] = Cstar 
params[7] = delV 
params[8] = delC 
params[9] = r_1 
params[10] = r_0 
params[11] = V_0
params[12] = C_0

C,V,f = mean_field_dynamics(params)

plt.plot(C,V)
print("Classification:", classify(V,params))
print("Conditions for SO phase met:", conditions)

plt.title("Stable Oscillations")
plt.xlabel("C")
plt.ylabel("V")
#plt.savefig("figures/step_SO.png")

Classification: 1
Conditions for SO phase met: False


Text(0,0.5,'V')

### Here I plot phase diagrams for the Step Function using a randomly connected matrix 

In [8]:
def simdyn4(M, l, params):
    return simDynamics4(M, l, params)

@jit(nopython=True)
def simDynamics4(M, l, params):  #params is a vector of parameters
    N=int(params[0])
    n=int(params[1])
    dt=params[2]
    tauv=params[3]
    tauc=params[4]
    Vstar=params[5]
    Cstar=params[6]
    delV=params[7]
    delC=params[8]
    r_1=params[9]
    V0=params[10]
    C0=params[11]
    delVmin=params[12]
    r_0=params[13]
    V=zeros(N*n).reshape(N,n)  #now we have potencial and calcium concentration for each neuron
    C=zeros(N*n).reshape(N,n)
    #P=zeros(n)  # function P(V), sigmoid
    P = zeros(N*n).reshape(N,n)
    deltaV=zeros(n)
    sump=zeros(n)  #sum of P over the vertices
    avV=zeros(N) #average potencial
    avC=zeros(N)  #average calcium
    firing_neurons_count = zeros(N*n).reshape(N,n) #Individually count how may neurons are actually firing in this model at a given step N.
    
    for i in range(n):
        V[0,i]=V0 #initial conditions
        C[0,i]=C0 
        if V[0,i] > Vstar: #keep track of and return how many neurons in the network are actually firing  
            firing_neurons_count[0,i] = 1
        avV[0]=n*V0  #we will divide to n later
        avC[0]=n*C0
    for i in range (N-1):
        for j in range(n):
            if (V[i,j]>Vstar): #V[i,j] stores voltage of each neuron instead of returning the average voltage 
                P[i,j]=r_1+r_0
            else:
                P[i,j]=r_0
            if (C[i,j]<Cstar):
                deltaV[j]=delV #change from 1.2 
            else:
                deltaV[j]=delVmin #changed from .4
        for j in range(n):
            a=int(l[j,0]) #a should give number of firing neighbors 
            for k in range(a):
                #b=int(l[j,k])
                b = int(l[j,k+1])
                sump[j]=sump[j]+P[i,b]  #if neuron connected to j-th neuron it participates in sum
        for j in range(n):
            V[i+1,j]=(-V[i,j]/tauv + deltaV[j]*sump[j])*dt + V[i,j]
            if V[i+1,j] > Vstar: 
                firing_neurons_count[i+1,j] = 1 #used to return count of firing neurons later
            #if ((firing_neurons_count[i,j] == 1) and (firing_neurons_count[i+1,j] == 0)):
            #    print(sump[j])
            C[i+1,j]=(-C[i,j]/tauc + delC*(r_1+r_0)*sump[j])*dt + C[i,j]
            avC[i+1]=avC[i+1]+C[i+1,j]
            avV[i+1]=avV[i+1]+V[i+1,j]
        for j in range(n):
            sump[j]=0
    for i in range (N):
        avC[i]=avC[i]/n
        avV[i]=avV[i]/n
        
        #plot([avC[i],avC[i+1]],[avV[i],avV[i+1]])    
    return(avC,avV, firing_neurons_count, P, V)

In [9]:
#delV cell 
N=10000
n=150 #150
dt=0.001
T=zeros(N)
for i in range(N):
    T[i]=i*dt
p = 1/6 #1/6 in real world 

M=getRandomConnectivity(n,p) #args(n,p)
l=matrixOfEdges(M,n)
M_prime = copy.deepcopy(M)
l_prime = copy.deepcopy(l)
to_cull = 0

to_cull = 0

tauv=0.01 #0.01
tauc=0.5 #0.5
Vstar=15 #Constant (according to experiments) 15
Cstar=20 #15
V0=18 #18
C0=18 #18

delV= 1.2 #1.2
delVmin = delV/3
delC= 0.015 #0.015
r_1= 70 #Max Firing rate is r_1 + r_0 
r_0 = 5 #Min firing rate 

#Instantiate array of parameters 
params=zeros(14)
params[0]=N
params[1]=n #instead of actual n, cull neurons til desired value
params[2]=dt
params[3]=tauv
params[4]=tauc
params[5]=Vstar
params[6]=Cstar
params[7]=delV
params[8]=delC
params[9]=r_1
params[10]=V0
params[11]=C0
params[12]=delVmin #connenctivity rate from above.
params[13]=r_0 

phasen = []
phaseV = []
kCore = []

dV = 0.1
delVmax = 30.


while True:
    C,V,f,P,Voltages = simdyn4(M_prime,l_prime,params)
    check = classify(V,params)
    while True:
        #import pdb; pdb.set_trace()
        params[1] -= 1
        #curr_n = curr_n-1
        to_cull += 1
        M_prime = killNeuron2(to_cull, M)
        l_prime = matrixOfEdges(M_prime, params[1])
        if params[1] <= 1:
            params[7] += dV
            params[12] = params[7]/3
            params[1] = n
            to_cull = 0
            M_prime = copy.deepcopy(M)
            l_prime = copy.deepcopy(l)
            #curr_n = n
            break
        C,V,f,P,Voltages = simdyn4(M_prime,l_prime,params)
        check2 = classify(V,params)
        if check2 != check:
            print("delV: ", params[7])
            print("transition from", check, "to", check2)
            #print("Kcore: ", largestKcore(M_prime), "Predicted:",floor(Vstar/(tauv*70*params[12])))
            print("Kcore: ", largestKcore(M_prime), "Predicted:",Vstar/(tauv*70*params[12]))
            phasen.append(params[1])
            phaseV.append(params[7])
            kCore.append(largestKcore(M_prime))
            params[7] += dV
            params[12] = params[7]/3
            if params[1] < n-3:
                to_cull -= 3
                params[1] += 3 #Changed from params[1] = n to track boundary 
                M_prime = killNeuron2(to_cull, M)
                l_prime = matrixOfEdges(M_prime, params[1])
            else:
                to_cull = 0
                params[1] = n 
                M_prime = copy.deepcopy(M)
                l_prime = copy.deepcopy(l)
                
            break

    if params[7] > delVmax:
        break
print('loop complete')


delV:  3.3000000000000016
transition from 3 to 2
Kcore:  18.0 Predicted: 19.48051948051947
delV:  3.4000000000000017
transition from 3 to 2
Kcore:  17.0 Predicted: 18.90756302521007
delV:  3.5000000000000018
transition from 3 to 2
Kcore:  17.0 Predicted: 18.3673469387755
delV:  3.600000000000002
transition from 3 to 2
Kcore:  16.0 Predicted: 17.857142857142847
delV:  3.700000000000002
transition from 3 to 2
Kcore:  16.0 Predicted: 17.374517374517364
delV:  3.800000000000002
transition from 3 to 2
Kcore:  15.0 Predicted: 16.9172932330827
delV:  3.900000000000002
transition from 3 to 2
Kcore:  15.0 Predicted: 16.48351648351647
delV:  4.000000000000002
transition from 3 to 2
Kcore:  14.0 Predicted: 16.071428571428562
delV:  4.100000000000001
transition from 3 to 2
Kcore:  14.0 Predicted: 15.679442508710794
delV:  4.200000000000001
transition from 3 to 2
Kcore:  14.0 Predicted: 15.306122448979586
delV:  4.300000000000001
transition from 3 to 2
Kcore:  13.0 Predicted: 14.950166112956806
del

delV:  12.69999999999997
transition from 3 to 2
Kcore:  4.0 Predicted: 5.061867266591688
delV:  12.79999999999997
transition from 3 to 2
Kcore:  4.0 Predicted: 5.0223214285714395
delV:  12.89999999999997
transition from 3 to 2
Kcore:  4.0 Predicted: 4.983388704318949
delV:  12.99999999999997
transition from 3 to 2
Kcore:  4.0 Predicted: 4.945054945054956
delV:  13.09999999999997
transition from 3 to 2
Kcore:  4.0 Predicted: 4.907306434024002
delV:  13.199999999999969
transition from 3 to 2
Kcore:  4.0 Predicted: 4.870129870129881
delV:  13.299999999999969
transition from 3 to 2
Kcore:  4.0 Predicted: 4.833512352309356
delV:  13.399999999999968
transition from 3 to 2
Kcore:  4.0 Predicted: 4.797441364605555
delV:  13.499999999999968
transition from 3 to 2
Kcore:  4.0 Predicted: 4.7619047619047725
delV:  13.599999999999968
transition from 3 to 2
Kcore:  4.0 Predicted: 4.726890756302532
delV:  13.699999999999967
transition from 3 to 2
Kcore:  4.0 Predicted: 4.692387904066747
delV:  13.799

delV:  22.40000000000005
transition from 3 to 2
Kcore:  2.0 Predicted: 2.869897959183667
delV:  22.50000000000005
transition from 3 to 2
Kcore:  2.0 Predicted: 2.8571428571428505
delV:  22.60000000000005
transition from 3 to 2
Kcore:  2.0 Predicted: 2.8445006321112447
delV:  22.700000000000053
transition from 3 to 2
Kcore:  2.0 Predicted: 2.8319697923222087
delV:  22.800000000000054
transition from 3 to 2
Kcore:  2.0 Predicted: 2.819548872180444
delV:  22.900000000000055
transition from 3 to 2
Kcore:  2.0 Predicted: 2.8072364316905736
delV:  23.000000000000057
transition from 3 to 2
Kcore:  2.0 Predicted: 2.7950310559006137
delV:  23.10000000000006
transition from 3 to 2
Kcore:  2.0 Predicted: 2.782931354359919
delV:  23.20000000000006
transition from 3 to 2
Kcore:  2.0 Predicted: 2.770935960591126
delV:  23.30000000000006
transition from 3 to 2
Kcore:  2.0 Predicted: 2.759043531575713
delV:  23.400000000000063
transition from 3 to 2
Kcore:  2.0 Predicted: 2.7472527472527397
delV:  23.

In [10]:
plot(phaseV, kCore)
plot(phaseV, phasen)